In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Preprocess data

In [3]:
import sys
#insert search path for packages
sys.path.insert(0, '../common/')
sys.path.insert(0, '../models/')
from importlib import reload

#project scripts
import util as util
import preprocessor as prep

In [4]:
training_set = pd.read_csv('../dataset/ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('../dataset/ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')


nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))


def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:, 1] [data[:, 0] == id_users]
        id_ratings = data[:, 2] [data[:, 0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

training_set = convert(training_set)
test_set = convert(test_set)

training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [5]:
class RBM():
    def __init__(self, nv, nh):
        #weights used for the probability of the visible nodes given the hidden nodes (p_v_given_h))
        # torch.rand : random normal distribution mean=0, variance=1 
        
        self.W = torch.randn(nh,nv)
        # bias probability of the hidden nodes given the visible nodes (p_h_given_v))
        # fake dimension for the batch = 1, to align with 2d tensors
        self.a = torch.randn(1,nh)
        # bias probability of the visible nodes is activated 
        #given the value of the hidden nodes (p_v_given_h))
        self.b = torch.randn(1, nv)
        # we can add more params like learning rate... in this section to be used in train()

    def sample_h(self, x):
        # probability h is activated given the value v is the sigmoid(Wx+a).
        # torch.mm make the product of two tensors. 
        # W.t()take the transpose because W is used for the p_v_given_h.
        wx = torch.mm(x, self.W.t())
        # .expand_as(wx) : expand the mini-batch.
        activation = wx + self.a.expand_as(wx)
        # probability p_h_given_v is the probability that the note drama genre is activated. 
        # v value is the input value. If v is a film drama, p_h_given_v will be hight. 
        # If v is not a film drama, p_h_given_v will be low.
        p_h_given_v = torch.sigmoid(activation)
        # Bernouilli RBM. we predict the user loves the movie or not (0 or 1).
        # activation or not activation of the nh neurons. 
        return p_h_given_v, torch.bernoulli(p_h_given_v)

    def sample_v(self, y): 
        # probability h is activated given the value v is the sigmoid(Wx+a).
        # torch.mm make the product of two tensors. 
        wy = torch.mm(y, self.W)
        # .expand_as(wx) : expand the mini-batch.
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        # Bernouilli RBM. we predict the user loves the movie or not (0 or 1).
        # activation or not activation of the nv neurons. 
        return p_v_given_h, torch.bernoulli(p_v_given_h)

        # Contrastive divergence Algorithm
        # Optimize the weights to minimize the energy.
        # ~ Maximize the Log-Likelihood of the model. 
        # Need to approximate the gradients with the algorithm contrastive divergence. 
    def train(self,v0,vk,ph0,phk):
        #ph0,phk regarding to 1 user, this train function
        # vk: visible nodes after k round trips of sampling
        # ph: ph0: vector prob at first iteration of hidden nodes = 1, given v0
        # phk vector prob at k iteration, for h = 1 given vk
        
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        # add ,0 for the tensor of two dimension 
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

        
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)


In [6]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [9]:
### Part 3 : Training the RBM 
nb_epoch = 10 
# upper bound is no included nb_epoch+1 

# First for loop : epoch for loop 
for epoch in range (1,nb_epoch+1):
    #loss function initialized to 0 at the beginning of the trainning 
    train_loss = 0
    # counter which is a float . 
    s = 0.

    # Second for loop : user forloop 
    # 0 lower bound 
    # nb_users-batch_size upper bound 
    # batch_size is the step of each batch (100)
    # First batch is from user id=0 ti user id =99
    
    #taking a batch of users
    for id_user in range(0,nb_users-batch_size,batch_size): #range(a,b,c)
        # at the beginning v0=vk 
        # vk is going to be updated
        # id_user,id_user+batch_size ~id_user+100
        
        vk=training_set[id_user:id_user+batch_size] #output of gibbs sampling, now dealing with specific user
        v0=training_set[id_user:id_user+batch_size]
        #initial probability prob hidden node at start = 1 given original ratings 
        ph0, _ = rbm.sample_h(v0)
        
        
        # k steps in the random walk
        # Third for loop : Contrastive divergence
        for k in range(10): #why 10? mcmc technique
            #call sample_h on visible nodes, get the first sampling of the first hidden node
            _,hk=rbm.sample_h(vk) #v0 is the target, will not change, we take vk
            #next we use the obtained hk to get the sampled vk
            _,vk=rbm.sample_v(hk) #update vk, second sample of the visible nodes, until all the k steps are sampled and updated vk
            
            #vk are visible nodes in the kstep random walk
            
            #above are the actual sampled values 
            
            # we don't want to learn where is no rating by the user
            # no update when -1 rating.
            vk[v0<0]=v0[v0<0] #training are not done on these -1 ratings
            
            phk,_=rbm.sample_h(vk) #get the probability of hk, last sample of the visible nodes in the kstep random walk, vk is the input, using this to get the phk
            
            # maximum likelihood to update the parameters, no return type
            rbm.train(v0,vk,ph0,phk)
            
            # Contrastive divergence to approximate the gradient
            
            # Compare vk updated after the training to v0 the target. 
            # simple distance in absolute value 
            # [vO>=0] take only the value with ratings / coherence with vk[v0<0]=[v0<0]
            #update the loss value
            
            # difference between target v0, and the prediction (last sample of the visible node of the contrastive divergence random walk)
            # mean absolute difference between truth and predictions
            train_loss+=torch.mean(torch.abs(v0[v0>=0]-vk[v0>=0]))
            
            #rmse version
            #train_loss+=np.sqrt(torch.mean((v0[v0>=0] - vk[v0>=0])**2)) # RMSE here
            s += 1.
    # in each epoch , see overall loss function        
    print('epoch: ' +str(epoch) +' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.2151)
epoch: 2 loss: tensor(0.2150)
epoch: 3 loss: tensor(0.2144)
epoch: 4 loss: tensor(0.2141)
epoch: 5 loss: tensor(0.2148)
epoch: 6 loss: tensor(0.2145)
epoch: 7 loss: tensor(0.2138)
epoch: 8 loss: tensor(0.2143)
epoch: 9 loss: tensor(0.2135)
epoch: 10 loss: tensor(0.2136)


In [10]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2460)
